In [87]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
#%cd /content/drive/My Drive/school/DSA3101/

/content/drive/My Drive/school/DSA3101


In [1]:
import pandas as pd

# Preparing scraped data for merge

In [ ]:
# scraped_df = pd.read_csv('final_data.csv')
# scraped_df = scraped_df.loc[:, ~scraped_df.columns.isin(['developer_response'])]
# scraped_df['row_id'] = scraped_df.index
# scraped_df = scraped_df[['row_id', 'date', 'client', 'bank', 'rating', 'review']]
# scraped_df

,row_id,date,client,bank,rating,review
0,0,2024-03-06 08:36:37,Apple,GXS,4.0,Great banking app with attractive interest rat...
1,1,2022-11-09 03:13:16,Apple,GXS,5.0,"A bank like no other, no bank have such amazin..."
2,2,2023-09-09 08:33:43,Apple,GXS,1.0,Notice that the drop in interest rate of 0.8% ...
3,3,2023-06-27 04:01:39,Apple,GXS,2.0,Sending money into my GXS account is a breeze ...
4,4,2022-10-02 04:17:49,Apple,GXS,5.0,I have to say that the UI/UX is one of the bes...
...,...,...,...,...,...,...
1089,1089,2023-12-26 10:31:11,Android,Trust,1.0,"After the latest update, I can't even open the..."
1090,1090,2023-12-26 03:54:34,Android,Trust,5.0,How to pay my bill for trust card
1091,1091,2023-12-23 13:20:31,Android,Trust,3.0,the app indicates exchange rate in trust is be...
1092,1092,2023-12-22 18:22:03,Android,Trust,1.0,The app is in conflict with my phone voice rec...


In [ ]:
# scraped_df1 = scraped_df.iloc[0:600, 1:]
# #scraped_df1.tail()
# scraped_df2 = scraped_df.iloc[696:1046, 1:]
# #scraped_df2.head()
# #scraped_df2.tail()
# final_scraped_df = pd.concat([scraped_df1, scraped_df2])
# #len(final_scraped_df)
# final_scraped_df.head()

,date,client,bank,rating,review
0,2024-03-06 08:36:37,Apple,GXS,4.0,Great banking app with attractive interest rat...
1,2022-11-09 03:13:16,Apple,GXS,5.0,"A bank like no other, no bank have such amazin..."
2,2023-09-09 08:33:43,Apple,GXS,1.0,Notice that the drop in interest rate of 0.8% ...
3,2023-06-27 04:01:39,Apple,GXS,2.0,Sending money into my GXS account is a breeze ...
4,2022-10-02 04:17:49,Apple,GXS,5.0,I have to say that the UI/UX is one of the bes...


# placeholder data

In [2]:
df = pd.read_csv("placeholder_data.csv")
df.head()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,1,2024-03-06,GXS,Apple,4,Negative,Praise,C,C1,Great banking app with attractive interest rat...
1,2,2022-11-09,GXS,Apple,5,Negative,Feedback,D,D1,"A bank like no other, no bank have such amazin..."
2,3,2023-09-09,GXS,Apple,1,Negative,Complaint,D,D3,Notice that the drop in interest rate of 0.8% ...
3,4,2023-06-27,GXS,Apple,2,Neutral,Feedback,D,D1,Sending money into my GXS account is a breeze ...
4,5,2022-10-02,GXS,Apple,5,Neutral,Praise,C,C1,I have to say that the UI/UX is one of the bes...


convert to datatype

In [3]:
preproc = df.copy()
df['date'] = pd.to_datetime(df['date'], errors='coerce')
preproc['bank'] = preproc['bank'].astype('category')
preproc['client'] = preproc['client'].astype('category')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
preproc['sentiment'] = preproc['sentiment'].astype('category')
preproc['intent'] = preproc['intent'].astype('category')
preproc['service'] = preproc['service'].astype('category')
preproc['issue'] = preproc['issue'].astype('category')
preproc.tail()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
1089,1090,2023-12-26,Trust,Android,1,Neutral,Feedback,A,A2,"After the latest update, I can't even open the..."
1090,1091,2023-12-26,Trust,Android,5,Positive,Feedback,C,C3,How to pay my bill for trust card
1091,1092,2023-12-23,Trust,Android,3,Positive,Feedback,C,C1,the app indicates exchange rate in trust is be...
1092,1093,2023-12-22,Trust,Android,1,Positive,Feedback,A,A1,The app is in conflict with my phone voice rec...
1093,1094,2023-12-22,Trust,Android,1,Negative,Feedback,D,D3,"Buggy app, keeps asking me to relogin and upon..."


In [ ]:
preproc.dtypes

id              int64
date           object
bank         category
client       category
rating          int64
sentiment    category
intent       category
service      category
issue        category
review         object
dtype: object

# 1. Join sentiment, intent, service, issue with scraped data

In [ ]:
# DONT RUN THIS CELL YET

# if row id = id of the original rows in scraped data
# merged_df = pd.merge(scraped_df, processed_df, on = 'row_id')
# merged_df = merged_df.loc[:, ~merged_df.columns.isin(['row_id'])]

# # if index of the two dataframes align
# merged_df =

# 2. Find top 2 issues within GXS

filter GXS data only

In [4]:
gxs = preproc[preproc['bank'] == 'GXS']
gxs.tail()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
890,891,2023-07-10,GXS,Android,1,Negative,Praise,D,D1,Have been downloading this app for past few da...
891,892,2023-07-09,GXS,Android,1,Positive,Complaint,B,B1,Misleading advertisement for flexiloan promoti...
892,893,2023-07-06,GXS,Android,1,Neutral,Complaint,A,A1,Can't even load properly. Prompted to set 6 di...
893,894,2023-07-04,GXS,Android,5,Neutral,Praise,A,A3,Easy to use. Awesome
894,895,2023-07-01,GXS,Android,2,Negative,Feedback,C,C3,I have merely tried to apply for your flexi lo...


filter negative sentiment

In [5]:
gxs_neg = gxs[gxs['sentiment'] == 'Negative']
# Reset the index if needed
gxs_neg.reset_index(drop=True, inplace=True)
gxs_neg.head()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,1,2024-03-06,GXS,Apple,4,Negative,Praise,C,C1,Great banking app with attractive interest rat...
1,2,2022-11-09,GXS,Apple,5,Negative,Feedback,D,D1,"A bank like no other, no bank have such amazin..."
2,3,2023-09-09,GXS,Apple,1,Negative,Complaint,D,D3,Notice that the drop in interest rate of 0.8% ...
3,10,2022-09-08,GXS,Apple,1,Negative,Feedback,A,A3,What a joke! Nothing happens after you input y...
4,11,2023-11-21,GXS,Apple,5,Negative,Feedback,D,D3,Honestly is my first time depositing digital b...


aggregate count of service

In [6]:
# Define a custom aggregation function that returns a DataFrame
def count_agg_function(group):
    # Get the group column value for this group
    group_value = group['service'].iloc[0]
    # Perform aggregation operations and return a DataFrame
    aggregated_df = pd.DataFrame({
        'service': [group_value],
        'count': [group['id'].count()]
    })
    return aggregated_df

In [7]:
# Aggregate the DataFrame by a specific column
gxs_neg_service_by_count = gxs_neg.groupby('service').apply(count_agg_function)
# Reset the index if needed
gxs_neg_service_by_count.reset_index(drop=True, inplace=True)

gxs_neg_service_by_count.head()

C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\3437183537.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gxs_neg_service_by_count = gxs_neg.groupby('service').apply(count_agg_function)
C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\3437183537.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gxs_neg_service_by_count = gxs_neg.groupby('service').apply(count_agg_function)


,service,count
0,A,21
1,B,26
2,C,29
3,D,28


get service 1 and 2

In [8]:
# Find the row with the maximum count in the 'count_column'
max_count_row = gxs_neg_service_by_count.loc[gxs_neg_service_by_count['count'].idxmax()]

# Get the variable with the most count
service_1 = max_count_row['service']

# Sort the aggregated DataFrame by 'count_column' in descending order
sorted_aggregated_df = gxs_neg_service_by_count.sort_values(by='count', ascending=False)

# Get the variable with the second highest count
service_2 = sorted_aggregated_df.iloc[1]['service']

print(service_1, service_2)

C D


filter service 1

In [9]:
service_1_df = gxs_neg[gxs_neg['service'] == service_1]
# Reset the index if needed
service_1_df.reset_index(drop=True, inplace=True)
service_1_df.head()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,1,2024-03-06,GXS,Apple,4,Negative,Praise,C,C1,Great banking app with attractive interest rat...
1,15,2022-10-31,GXS,Apple,5,Negative,Praise,C,C1,Despite a lengthy onboarding but my account go...
2,24,2023-11-11,GXS,Apple,5,Negative,Praise,C,C1,Can try for saving money !! My money said they...
3,28,2022-12-07,GXS,Apple,5,Negative,Praise,C,C2,the app is easy to navigate and the UI/UX is e...
4,29,2022-12-09,GXS,Apple,5,Negative,Feedback,C,C2,My fav feature is the saving pockets can't wai...


aggregate count of service 1

In [10]:
# Define a custom aggregation function that returns a DataFrame
def issue_count_agg_function(group):
    # Initialize an empty list to store the results
    results = []

    # Iterate over all the values in the 'issue' column for this group
    for group_value in group['issue'].unique():
        # Check if group_value is not None
        if pd.notna(group_value):
            # Perform aggregation operations and append the result to the list
            aggregated_result = {
                'issue': group_value,
                'count': group.loc[group['issue'] == group_value, 'id'].count()
                # Add more aggregation operations as needed
            }
            results.append(aggregated_result)

    # Convert the list of results to a DataFrame
    aggregated_df = pd.DataFrame(results)

    return aggregated_df


In [11]:
# Aggregate the DataFrame by a specific column
gxs_neg_issue_by_count_1 = service_1_df.groupby('issue').apply(issue_count_agg_function).dropna()
# Reset the index if needed
gxs_neg_issue_by_count_1.reset_index(drop=True, inplace=True)

gxs_neg_issue_by_count_1.head()

C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\2798398168.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gxs_neg_issue_by_count_1 = service_1_df.groupby('issue').apply(issue_count_agg_function).dropna()
C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\2798398168.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gxs_neg_issue_by_count_1 = service_1_df.groupby('issue').apply(issue_count_agg_function).dropna()


,issue,count
0,C1,10.0
1,C2,9.0
2,C3,10.0


get issue 1

In [12]:
# Find the row with the maximum count in the 'count_column'
max_count_row_issue = gxs_neg_issue_by_count_1.loc[gxs_neg_issue_by_count_1['count'].idxmax()]

# Get the variable with the most count
issue_1 = max_count_row_issue['issue']
issue_1

'C1'

filter service 2

In [13]:
service_2_df = gxs_neg[gxs_neg['service'] == service_2]
# Reset the index if needed
service_2_df.reset_index(drop=True, inplace=True)
service_2_df.head()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,2,2022-11-09,GXS,Apple,5,Negative,Feedback,D,D1,"A bank like no other, no bank have such amazin..."
1,3,2023-09-09,GXS,Apple,1,Negative,Complaint,D,D3,Notice that the drop in interest rate of 0.8% ...
2,11,2023-11-21,GXS,Apple,5,Negative,Feedback,D,D3,Honestly is my first time depositing digital b...
3,22,2024-03-19,GXS,Apple,5,Negative,Feedback,D,D3,Very smooth to use and attractive app. Simple ...
4,40,2023-07-25,GXS,Apple,1,Negative,Feedback,D,D1,Noticed the news writing they offer 3.48% inte...


aggregate count of service 2

In [14]:
# Aggregate the DataFrame by a specific column
gxs_neg_issue_by_count_2 = service_2_df.groupby('issue').apply(issue_count_agg_function).dropna()
# Reset the index if needed
gxs_neg_issue_by_count_2.reset_index(drop=True, inplace=True)

gxs_neg_issue_by_count_2.head()

C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\2382357473.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gxs_neg_issue_by_count_2 = service_2_df.groupby('issue').apply(issue_count_agg_function).dropna()
C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\2382357473.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gxs_neg_issue_by_count_2 = service_2_df.groupby('issue').apply(issue_count_agg_function).dropna()


,issue,count
0,D1,11.0
1,D2,4.0
2,D3,13.0


get issue 2

In [15]:
# Find the row with the maximum count in the 'count_column'
max_count_row_issue_2 = gxs_neg_issue_by_count_2.loc[gxs_neg_issue_by_count_2['count'].idxmax()]

# Get the variable with the most count
issue_2 = max_count_row_issue_2['issue']
issue_2

'D3'

# 2a. Issue #1

filter issue 1

In [35]:
issue_1_df = gxs_neg[gxs_neg['issue'] == issue_1]
# Reset the index if needed
issue_1_df.reset_index(drop=True, inplace=True)
issue_1_df

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,1,2024-03-06,GXS,Apple,4,Negative,Praise,C,C1,Great banking app with attractive interest rat...
1,15,2022-10-31,GXS,Apple,5,Negative,Praise,C,C1,Despite a lengthy onboarding but my account go...
2,24,2023-11-11,GXS,Apple,5,Negative,Praise,C,C1,Can try for saving money !! My money said they...
3,120,2022-12-30,GXS,Apple,5,Negative,Praise,C,C1,Great features! Love it!
4,719,2024-01-12,GXS,Android,5,Negative,Praise,C,C1,I don't set. A password in my phone n not allo...
5,824,2023-08-07,GXS,Android,1,Negative,Feedback,C,C1,"Barely a month in, GXS has indeed chosen to sl..."
6,829,2023-08-07,GXS,Android,5,Negative,Complaint,C,C1,Seamless. Got it up and running in less than 5...
7,855,2023-07-24,GXS,Android,1,Negative,Praise,C,C1,"Everytime successfully signec up, after log in..."
8,860,2023-07-22,GXS,Android,5,Negative,Feedback,C,C1,Seamless registration. All done in less than 1...
9,867,2023-07-20,GXS,Android,5,Negative,Praise,C,C1,Nice


give filtered table with rowid, text data of issue 1 for recommendation

In [36]:
issue_1_df['recommendation_no'] = 'R1'
issue_1_recommend = issue_1_df[['id', 'service', 'issue','recommendation_no','review']]
issue_1_recommend

C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\2843638045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issue_1_df['recommendation_no'] = 'R1'


,id,service,issue,recommendation_no,review
0,1,C,C1,R1,Great banking app with attractive interest rat...
1,15,C,C1,R1,Despite a lengthy onboarding but my account go...
2,24,C,C1,R1,Can try for saving money !! My money said they...
3,120,C,C1,R1,Great features! Love it!
4,719,C,C1,R1,I don't set. A password in my phone n not allo...
5,824,C,C1,R1,"Barely a month in, GXS has indeed chosen to sl..."
6,829,C,C1,R1,Seamless. Got it up and running in less than 5...
7,855,C,C1,R1,"Everytime successfully signec up, after log in..."
8,860,C,C1,R1,Seamless registration. All done in less than 1...
9,867,C,C1,R1,Nice


# 2b. Issue #2

filter issue 2

In [38]:
issue_2_df = gxs_neg[gxs_neg['issue'] == issue_2]
# Reset the index if needed
issue_2_df.reset_index(drop=True, inplace=True)
issue_2_df

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,3,2023-09-09,GXS,Apple,1,Negative,Complaint,D,D3,Notice that the drop in interest rate of 0.8% ...
1,11,2023-11-21,GXS,Apple,5,Negative,Feedback,D,D3,Honestly is my first time depositing digital b...
2,22,2024-03-19,GXS,Apple,5,Negative,Feedback,D,D3,Very smooth to use and attractive app. Simple ...
3,50,2023-06-05,GXS,Apple,5,Negative,Feedback,D,D3,Easy to use and log in! Very clean instruction.
4,60,2023-08-07,GXS,Apple,1,Negative,Complaint,D,D3,GXS touted a 3.48% interest in their savings p...
5,67,2022-09-05,GXS,Apple,5,Negative,Praise,D,D3,Nice design!
6,80,2023-05-02,GXS,Apple,1,Negative,Feedback,D,D3,App keeps giving me “something went wrong” and...
7,84,2024-03-15,GXS,Apple,1,Negative,Complaint,D,D3,Apply one year ago..say when slots open up wil...
8,98,2024-01-01,GXS,Apple,5,Negative,Complaint,D,D3,Good to use
9,129,2022-09-16,GXS,Apple,5,Negative,Praise,D,D3,🚀🚀🚀🚀🚀


give filtered table with rowid, text data of issue 2 for recommendation

In [39]:
issue_2_df['recommendation_no'] = 'R2'
issue_2_recommend = issue_2_df[['id', 'service', 'issue', 'recommendation_no', 'review']]
issue_2_recommend

C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\4050527689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issue_2_df['recommendation_no'] = 'R2'


,id,service,issue,recommendation_no,review
0,3,D,D3,R2,Notice that the drop in interest rate of 0.8% ...
1,11,D,D3,R2,Honestly is my first time depositing digital b...
2,22,D,D3,R2,Very smooth to use and attractive app. Simple ...
3,50,D,D3,R2,Easy to use and log in! Very clean instruction.
4,60,D,D3,R2,GXS touted a 3.48% interest in their savings p...
5,67,D,D3,R2,Nice design!
6,80,D,D3,R2,App keeps giving me “something went wrong” and...
7,84,D,D3,R2,Apply one year ago..say when slots open up wil...
8,98,D,D3,R2,Good to use
9,129,D,D3,R2,🚀🚀🚀🚀🚀


# 3. Find top issue to keep up with/continue leading Trust Bank

filter Trust bank data only

In [20]:
trust = preproc[preproc['bank'] == 'Trust']
trust.reset_index(drop=True, inplace=True)
trust.head()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,133,2022-11-08,Trust,Apple,5,Positive,Praise,D,D1,"I had my card locked the night before, and con..."
1,134,2023-02-11,Trust,Apple,5,Negative,Feedback,C,C1,"Been using it for 3 months, everything’s great..."
2,135,2022-09-07,Trust,Apple,1,Positive,Feedback,B,B2,"On 04Sept, i accidentally clicked “yes” to be ..."
3,136,2023-08-22,Trust,Apple,1,Positive,Complaint,A,A2,The interface is not intuitive at all. You lit...
4,137,2023-02-15,Trust,Apple,5,Neutral,Feedback,C,C3,"Hi, I’ve been using trust bank for a while now..."


filter positive sentiment

In [21]:
trust_pos = trust[trust['sentiment'] == 'Positive']
trust_pos.reset_index(drop=True, inplace=True)
trust_pos.head()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,133,2022-11-08,Trust,Apple,5,Positive,Praise,D,D1,"I had my card locked the night before, and con..."
1,135,2022-09-07,Trust,Apple,1,Positive,Feedback,B,B2,"On 04Sept, i accidentally clicked “yes” to be ..."
2,136,2023-08-22,Trust,Apple,1,Positive,Complaint,A,A2,The interface is not intuitive at all. You lit...
3,140,2023-06-12,Trust,Apple,1,Positive,Complaint,D,D2,Now that I’m happy to use the Trust Card for m...
4,145,2023-07-06,Trust,Apple,1,Positive,Feedback,D,D2,I'm served by Navind to waive off Purchase Int...


aggregate count of service

In [22]:
# Aggregate the DataFrame by a specific column
trust_pos_service_by_count = trust_pos.groupby('service').apply(count_agg_function)
# Reset the index if needed
trust_pos_service_by_count.reset_index(drop=True, inplace=True)

trust_pos_service_by_count.head()

C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\3550245484.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trust_pos_service_by_count = trust_pos.groupby('service').apply(count_agg_function)
C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\3550245484.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trust_pos_service_by_count = trust_pos.groupby('service').apply(count_agg_function)


,service,count
0,A,63
1,B,49
2,C,49
3,D,64


In [23]:
# Find the row with the maximum count in the 'count_column'
max_count_row_service_3 = trust_pos_service_by_count.loc[trust_pos_service_by_count['count'].idxmax()]

# Get the variable with the most count
service_3 = max_count_row_service_3['service']
service_3

'D'

filter service 3

In [24]:
service_3_df = trust_pos[trust_pos['service'] == service_3]
# Reset the index if needed
service_3_df.reset_index(drop=True, inplace=True)
service_3_df.head()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,133,2022-11-08,Trust,Apple,5,Positive,Praise,D,D1,"I had my card locked the night before, and con..."
1,140,2023-06-12,Trust,Apple,1,Positive,Complaint,D,D2,Now that I’m happy to use the Trust Card for m...
2,145,2023-07-06,Trust,Apple,1,Positive,Feedback,D,D2,I'm served by Navind to waive off Purchase Int...
3,159,2022-09-19,Trust,Apple,4,Positive,Complaint,D,D3,While most of the features right now are clear...
4,190,2022-09-05,Trust,Apple,5,Positive,Complaint,D,D3,"After i submit my info , i am stucked at the n..."


aggregate count of service 3

In [25]:
# Aggregate the DataFrame by a specific column
trust_pos_issue_by_count = service_3_df.groupby('issue').apply(issue_count_agg_function).dropna()
# Reset the index if needed
trust_pos_issue_by_count.reset_index(drop=True, inplace=True)

trust_pos_issue_by_count.head()

C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\3892870672.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trust_pos_issue_by_count = service_3_df.groupby('issue').apply(issue_count_agg_function).dropna()
C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\3892870672.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trust_pos_issue_by_count = service_3_df.groupby('issue').apply(issue_count_agg_function).dropna()


,issue,count
0,D1,16.0
1,D2,23.0
2,D3,25.0


get issue 3

In [26]:
# Find the row with the maximum count in the 'count_column'
max_count_row_issue_3 = trust_pos_issue_by_count.loc[trust_pos_issue_by_count['count'].idxmax()]

# Get the variable with the most count
issue_3 = max_count_row_issue_3['issue']
issue_3

'D3'

get reviews with issue 3 in gxs

In [27]:
issue_3_df = gxs[gxs['issue'] == issue_3]
# Reset the index if needed
issue_3_df.reset_index(drop=True, inplace=True)
issue_3_df.head()

,id,date,bank,client,rating,sentiment,intent,service,issue,review
0,3,2023-09-09,GXS,Apple,1,Negative,Complaint,D,D3,Notice that the drop in interest rate of 0.8% ...
1,11,2023-11-21,GXS,Apple,5,Negative,Feedback,D,D3,Honestly is my first time depositing digital b...
2,22,2024-03-19,GXS,Apple,5,Negative,Feedback,D,D3,Very smooth to use and attractive app. Simple ...
3,50,2023-06-05,GXS,Apple,5,Negative,Feedback,D,D3,Easy to use and log in! Very clean instruction.
4,60,2023-08-07,GXS,Apple,1,Negative,Complaint,D,D3,GXS touted a 3.48% interest in their savings p...
5,67,2022-09-05,GXS,Apple,5,Negative,Praise,D,D3,Nice design!
6,80,2023-05-02,GXS,Apple,1,Negative,Feedback,D,D3,App keeps giving me “something went wrong” and...
7,84,2024-03-15,GXS,Apple,1,Negative,Complaint,D,D3,Apply one year ago..say when slots open up wil...
8,98,2024-01-01,GXS,Apple,5,Negative,Complaint,D,D3,Good to use
9,112,2022-12-07,GXS,Apple,5,Neutral,Feedback,D,D3,"app is really easy to navigate, user-friendly"


give filtered table with rowid, text data of issue 3 for recommendation

In [40]:
issue_3_df['recommendation_no'] = 'R3'
issue_3_recommend = issue_3_df[['id', 'service', 'issue','recommendation_no','review']]
issue_3_recommend.head()

C:\Users\caleb\AppData\Local\Temp\ipykernel_97624\1393762953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issue_3_df['recommendation_no'] = 'R3'


,id,service,issue,recommendation_no,review
0,3,D,D3,R3,Notice that the drop in interest rate of 0.8% ...
1,11,D,D3,R3,Honestly is my first time depositing digital b...
2,22,D,D3,R3,Very smooth to use and attractive app. Simple ...
3,50,D,D3,R3,Easy to use and log in! Very clean instruction.
4,60,D,D3,R3,GXS touted a 3.48% interest in their savings p...


# 4. Merge the 3 dfs for recommendation

In [41]:
combine_issue_recommend = pd.concat([issue_1_recommend, issue_2_recommend, issue_3_recommend])
combine_issue_recommend.reset_index(drop=True, inplace=True)
combine_issue_recommend

,id,service,issue,recommendation_no,review
0,1,C,C1,R1,Great banking app with attractive interest rat...
1,15,C,C1,R1,Despite a lengthy onboarding but my account go...
2,24,C,C1,R1,Can try for saving money !! My money said they...
3,120,C,C1,R1,Great features! Love it!
4,719,C,C1,R1,I don't set. A password in my phone n not allo...
5,824,C,C1,R1,"Barely a month in, GXS has indeed chosen to sl..."
6,829,C,C1,R1,Seamless. Got it up and running in less than 5...
7,855,C,C1,R1,"Everytime successfully signec up, after log in..."
8,860,C,C1,R1,Seamless registration. All done in less than 1...
9,867,C,C1,R1,Nice


export to csv

In [42]:
combine_issue_recommend.to_csv("eda_res.csv", index = False)